# Import dependencies and cleaning data

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
import pandas as pd
import os
from sklearn import tree
from sklearn.model_selection import GridSearchCV

In [ ]:
data = pd.read_csv('germanHeaders.csv', header=None)

In [ ]:
columns = data.iloc[0].tolist()

In [ ]:
data.columns = columns

In [ ]:
data = data.iloc[1:]

In [ ]:
data = data.iloc[:,1:]

In [ ]:
data.columns.tolist()

In [ ]:
data

In [ ]:
a_columns = ['account_status', 'credit_history', 'purpose', 'savings_account', 'unemployed',
     'personal_status_sex', 'debtors/guarantors', 'property', 'other_installment_plans', 'housing', 'job', 'telephone', 'foreign_worker']

In [ ]:
data[a_columns] = data[a_columns].replace({'A':''}, regex=True)
data['good/bad'] = data['good/bad'].replace({'2':'0'}, regex=True)

In [ ]:
data[a_columns] = data[a_columns].apply(pd.to_numeric) 
data

In [ ]:
#data.to_csv('cleandata_woutcome.csv')

# Assign variables and adjust sampling

In [ ]:
X = data.drop("good/bad", axis=1)
y = data["good/bad"]
attribute_names = X.columns
sc = StandardScaler()
pca = decomposition.PCA()
#X.head()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1)

In [ ]:
y_train.value_counts()

# Decision Tree

In [ ]:
dtc = tree.DecisionTreeClassifier(random_state=1)
dtc.fit(X_train, y_train)

#y_pred_class = dtc.predict(X_test)
dtc.score(X_test, y_test)

In [ ]:
# test different parameters

sample_split_range = list(range(1,200))
param_grid = dict(min_samples_split=sample_split_range)

grid = GridSearchCV(dtc, param_grid, verbose=3)
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
predictions = grid.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(y_test, predictions,
                            target_names=["bad","good"]))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=2000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, attribute_names),reverse=True)